# Reconnaissance faciale via $k$ plus proches voisins ($k$-NN)

>## Partie 2: Reconnaissance faciale en temps réel

## Librairies

In [18]:
import pickle
import numpy as np

import cv2

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

## Algorithme

In [19]:
with open('data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)

with open('data/noms.pkl', 'rb') as fh:
    noms = pickle.load(fh)

print('Shape of visages matrix --> ', visages.shape)

Shape of visages matrix -->  (20, 50, 50, 3)


In [20]:
# Supposons que `noms` est votre liste d'étiquettes textuelles
label_encoder = LabelEncoder()
noms_integers = label_encoder.fit_transform(noms)  # Convertit en entiers
# Supposons que vous avez 3 visages connus + 1 classe "Inconnu" = 4 classes au total
nombre_classes = len(np.unique(noms_integers))

N = len(noms)
visages = visages.reshape(N, -1)

# Construction du modèle CNN pour classification multi-classes
modele_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(nombre_classes, activation='softmax')  # Utilisez softmax pour la classification multi-classes
])

modele_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
visages_nn = visages.reshape(-1, 50, 50, 3) / 255.0

# Assurez-vous que vos étiquettes (noms_binaires) sont encodées en One-Hot
noms_binaires_one_hot = to_categorical(noms_integers, num_classes=nombre_classes)

# Entraînement du modèle CNN pour classification multi-classes
modele_cnn.fit(visages_nn, noms_binaires_one_hot, epochs=10)


ValueError: invalid literal for int() with base 10: 'Yanis'

## Exécution

In [ ]:
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) # 0 pour 'built-in' caméra, 1 pour caméra externe

while True:
    
    ret, trame = camera.read()
    if ret == True:
        
        gris = cv2.cvtColor(trame, cv2.COLOR_BGR2GRAY)
        coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

        for (x, y, l, h) in coordonnees_visage:
            
            visage = trame[y:y + h, x:x + l, :]
            visage_redimensionne = cv2.resize(visage, (50, 50)).flatten().reshape(1,-1)
            
            texte = tree.predict(visage_redimensionne)
            
            cv2.putText(trame, texte[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(trame, (x, y), (x + l, y + l), (0, 0, 255), 2)

        cv2.imshow('Reconnaissance faciale en temps réel', trame)
        
        if cv2.waitKey(1) == 27:
            break
            
    else:
        
        print("erreur")
        break

cv2.destroyAllWindows()
camera.release()

**Exemple d'exécution du système...**

<img src="moi.png" width="400" height="auto" />